In [1]:
import random
import numpy as np
import math

In [2]:
pop_size = 12
valid_genes = "01"
length_of_chromosome = 12
no_of_training_rules=4
rule_len = 3

In [3]:
def train(fitness_threshold, p,r,m):
    population = initialise_population(p) #np array
    print("initial population: ",population)
    fitness = compute_fitness(population)  #np array
    max_fitness = (max(fitness))
    prob = [fit/sum(fitness) for fit in fitness]
    i=0
    while(max_fitness <fitness_threshold):
        #selection
        new_pop = np.random.choice(population, size=int((1-r)*p), p = prob, replace=False)
        print(new_pop)
        offsprings = crossover(population, r,p,prob)
        new_pop = np.append(new_pop, offsprings) #append
        p= len(new_pop)
        new_pop = mutate(new_pop, m,p)
        population = new_pop #update
        print("population in itr ",i," : ",population)
        fitness = compute_fitness(population)
        max_fitness = (max(fitness))
        prob = [fit/sum(fitness) for fit in fitness] #probabilities
        print("max fitness in itr ",i," : ",max_fitness)
        i=i+1
    ind = np.argmax(fitness)
    max_hypothesis = population[ind]
    print("max_fitness: ",max_fitness)
    return max_hypothesis,max_fitness

In [4]:
def initialise_population(p):
    population=[]
    for j in range(pop_size):
        s=""
        for i in range(p):
            temp = str(random.randint(0, 1))
            s += temp
        population.append(s)
    return np.array(population)

In [5]:
def compute_fitness(pop):
    fitness =[]
    for i in range(pop.size):
        h = pop[i]
        score = 0
        j=0
        r1=0
        r2=0
        r3=0
        r4=0
        while(j<len(h)):
            if h[j]=='0' and h[j+1]=='0' and h[j+2]=='0':
                r1=1
            elif h[j]=='0' and h[j+1]=='1' and h[j+2]=='1':
                r2=1
            elif h[j]=='1' and h[j+1]=='0' and h[j+2]=='1':
                r3=1
            elif h[j]=='1' and h[j+1]=='1' and h[j+2]=='0':
                r4=1
            j=j+3
        score = ((r1+r2+r3+r4)/no_of_training_rules)**2
        fitness.append(score)
    return np.array(fitness)        

In [6]:
def crossover(pop,r,p,prob):
    n= math.ceil((r*p)/2) #check this
    parents_chosen = np.random.choice(pop, size=n*2, p = prob, replace=False)
    print(parents_chosen)
    offsprings =[]
    j=0
    while(j <len(parents_chosen)):
        parent1 = parents_chosen[j]
        parent2 = parents_chosen[j+1]
        p1, p2 = gen_crossover_points(len(parent1)+0.99,1) # gabil's method
        d1 = p1 % rule_len
        d2 = p2 % rule_len
        if(d1==0):
            d1=3
        if(d2==0):
            d2=3
        while(1):
            p3, p4 = gen_crossover_points(len(parent2)+0.99,1)
            d3 = p3 % rule_len
            d4 = p4 % rule_len
            if(d3==0):
                d3=3
            if(d4==0):
                d4=3
            if(d1 == d3 and d2 ==d4):
                break
        os1 = parent1[0:p1] + parent2[p3:p4] + parent1[p2:len(parent1)]
        os2 = parent2[0:p3] + parent1[p1:p2] + parent2[p4 : len(parent2)]
        offsprings.append(os1)
        offsprings.append(os2)
        j=j+2
    return np.array(offsprings)

In [7]:
def gen_crossover_points(hi,lo=1):
    pt1 = int(np.random.uniform(low=lo, high=hi))
    pt2 = int(np.random.uniform(low=lo, high=hi))
    while pt1 == pt2:
        pt2 = int(np.random.uniform(low=lo, high=hi))
    p1, p2 = min(pt1,pt2), max(pt1,pt2)
    return p1,p2

In [8]:
def mutate(pop, m,p):
    n = int(m*p)
    to_mutate = np.random.choice(len(pop)-1, size=n, replace=False)
    for p in to_mutate:
        ch = pop[p]
        k = int(np.random.uniform(low=0,high=len(ch)-1))
        if(ch[k]=='0'):
            ch = ch[:k]+"1"+ch[k+1:]
            #ch[k]='1'
        else:
            ch = ch[:k]+"0"+ch[k+1:]
        pop[p]=ch
    return pop

In [9]:
hyp,fit = train(1, 12, 0.3, 0.2)
print(hyp, fit)

initial population:  ['001001010011' '000010111000' '110011001110' '011001101110'
 '011010011011' '110110111000' '110010010000' '101110111101'
 '100110100011' '010100001011' '100101111110' '110011001110']
['101110111101' '011010011011' '110010010000' '100110100011'
 '110110111000' '110011001110' '100101111110' '011001101110']
['110011001110' '101110111101' '110011001110' '011010011011']
population in itr  0  :  ['101110111101' '011010011011' '110010010000' '100010100011'
 '110110111000' '110011001110' '110101111110' '011001101110'
 '110011001110' '101110111101' '110011010010' '011001111011']
max fitness in itr  0  :  0.5625
['110011001110' '110011010010' '101110111101' '110110111000'
 '011001101110' '101110111101' '110011001110' '110010010000']
['011001101110' '011010011011' '110101111110' '110011010010']
population in itr  1  :  ['110111001110' '110011010010' '101110111101' '110110111000'
 '011001101110' '101110111101' '110011001110' '110010010000' '001011110'
 '011010011001101' '1101